In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('rec').getOrCreate()

24/04/07 11:33:03 WARN Utils: Your hostname, Luiss-Mac-mini.local resolves to a loopback address: 127.0.0.1; using 192.168.100.206 instead (on interface en1)
24/04/07 11:33:03 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/07 11:33:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/04/07 11:33:03 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/04/07 11:33:03 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
24/04/07 11:33:03 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
24/04/07 11:33:03 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
24/04/07 11:33:03 WARN Utils: Service 'SparkUI' could not bind on port 4044. Attempti

In [3]:
from pyspark.ml.recommendation import ALS

In [4]:
from pyspark.ml.evaluation import RegressionEvaluator

In [5]:
data = spark.read.csv('movielens_ratings.csv', header=True, inferSchema=True)

In [6]:
data.show()

+-------+------+------+
|movieId|rating|userId|
+-------+------+------+
|      2|   3.0|     0|
|      3|   1.0|     0|
|      5|   2.0|     0|
|      9|   4.0|     0|
|     11|   1.0|     0|
|     12|   2.0|     0|
|     15|   1.0|     0|
|     17|   1.0|     0|
|     19|   1.0|     0|
|     21|   1.0|     0|
|     23|   1.0|     0|
|     26|   3.0|     0|
|     27|   1.0|     0|
|     28|   1.0|     0|
|     29|   1.0|     0|
|     30|   1.0|     0|
|     31|   1.0|     0|
|     34|   1.0|     0|
|     37|   1.0|     0|
|     41|   2.0|     0|
+-------+------+------+
only showing top 20 rows



In [13]:
data.describe().show()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



In [14]:
training, test = data.randomSplit([0.8, 0.2])

In [15]:
als = ALS(maxIter=5, regParam=0.01, userCol='userId', itemCol='movieId', ratingCol='rating')

In [17]:
model = als.fit(training)

24/04/07 11:40:19 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
24/04/07 11:40:19 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [18]:
predictions = model.transform(test)

In [19]:
predictions.show()

+-------+------+------+-----------+
|movieId|rating|userId| prediction|
+-------+------+------+-----------+
|      1|   1.0|    28|  4.0747304|
|      7|   1.0|    28| 0.13834327|
|      1|   1.0|    26|  1.9950837|
|      2|   2.0|     1|  2.4844139|
|      3|   1.0|    13|  1.1179471|
|      2|   1.0|    16|   1.266698|
|      2|   1.0|     3| 0.21429549|
|      0|   1.0|    20| 0.44031447|
|      4|   1.0|     5|-0.05637028|
|      5|   1.0|     9|  1.7601011|
|      6|   1.0|     9|  0.6047336|
|      0|   1.0|     8| 0.32383573|
|      2|   1.0|    23| -1.2743067|
|      1|   1.0|     7| -0.7657709|
|      2|   2.0|     7|  1.6225964|
|      5|   1.0|    29|   0.527334|
|      0|   1.0|    21|  0.6631827|
|      2|   4.0|    21|  2.1997173|
|      6|   2.0|    11|  2.6362674|
|      1|   1.0|    14|   1.096085|
+-------+------+------+-----------+
only showing top 20 rows



In [20]:
evaluator = RegressionEvaluator(metricName='rmse', labelCol='rating', predictionCol='prediction')

In [22]:
rmse = evaluator.evaluate(predictions)

In [23]:
print(rmse)

1.739704900360535


In [28]:
single_user = test.filter(test['userId']==11).select(['movieId', 'userId'])

In [29]:
single_user.show()

+-------+------+
|movieId|userId|
+-------+------+
|      6|    11|
|     38|    11|
|     41|    11|
|     47|    11|
|     48|    11|
|     62|    11|
|     64|    11|
|     81|    11|
|     97|    11|
|     99|    11|
+-------+------+



In [30]:
recommendations = model.transform(single_user)

In [33]:
recommendations.orderBy('prediction', ascending=False).show()

+-------+------+-----------+
|movieId|userId| prediction|
+-------+------+-----------+
|     38|    11|  2.7593203|
|      6|    11|  2.6362674|
|     48|    11|  1.8970301|
|     81|    11|   1.868419|
|     99|    11|  1.8380835|
|     62|    11|   1.774585|
|     41|    11|   0.524067|
|     47|    11| 0.36801693|
|     97|    11|-0.30641323|
|     64|    11| -3.1391876|
+-------+------+-----------+

